### geo6_inf_counts

- sjoin한 결과와 286개의 상권 데이터를 이용해 상권별 확진자수 도출
- 프로젝트 목표에 맞게 분기별/상권별 확진자수를 새로 도출
- folium choropleth를 이용하여 상권별 확진자수 시각화

In [2]:
import folium
import json
from geopandas import GeoSeries, GeoDataFrame, read_file
from shapely.geometry import Point, Polygon
import geopandas as gpd

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
import pandas as pd
import numpy as np

### sjoin한 결과 가져오기

In [5]:
inf_in_poly = gpd.read_file('./data/inf_in_poly.geojson')
inf_in_poly.head()

,확진장소,index_right,상권명,geometry
0,긱스타PC방,1387,서울 관악구 서울대입구역_1,POINT (126.95211 37.47986)
1,에그드랍서울대입구점,1387,서울 관악구 서울대입구역_1,POINT (126.95289 37.48021)
2,서울대입구역2호선,1387,서울 관악구 서울대입구역_1,POINT (126.95271 37.48128)
3,서울대입구역2호선,1387,서울 관악구 서울대입구역_1,POINT (126.95271 37.48128)
4,스타버스코인노래연습장서울대본점,1387,서울 관악구 서울대입구역_1,POINT (126.95298 37.48067)


In [6]:
inf_in_poly.nunique()

확진장소           606
index_right    286
상권명            286
geometry       585
dtype: int64

In [7]:
inf_in_poly['상권명'].value_counts()

이태원 관광특구           646
홍익대학교 주변           536
서울 관악구 서울대입구역_1    482
학동로56길             471
용산구청               353
                  ... 
남부순환로97길             6
영천시장                 6
남성사계시장               6
영등포전통시장              6
코엑스                  6
Name: 상권명, Length: 286, dtype: int64

In [7]:
# inf_in_poly=inf_in_poly.to_crs(epsg=3857) 
# sjoin을 하는게 아니니까 굳이 안해줘도 됨.

### 확진자가 포함된 상권을 이용해서 각 상권에 포함되는 확진자 수 구하기

In [75]:
df_poly_286 = gpd.read_file('./data/df_poly_286.geojson')
df_poly_286

,상권명,geometry
0,서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9..."
1,서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2..."
2,서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6..."
3,호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9..."
4,서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8..."
...,...,...
281,서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3..."
282,강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3..."
283,모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5..."
284,석촌시장,"POLYGON ((14149657.028 4509182.741, 14149713.8..."


In [10]:
inf_in_poly['상권명'].value_counts()

이태원 관광특구           646
홍익대학교 주변           536
서울 관악구 서울대입구역_1    482
학동로56길             471
용산구청               353
                  ... 
남부순환로97길             6
영천시장                 6
남성사계시장               6
영등포전통시장              6
코엑스                  6
Name: 상권명, Length: 286, dtype: int64

In [76]:
df_poly_286['확진자수'] = inf_in_poly['상권명'].value_counts()
df_poly_286 # 실패

,상권명,geometry,확진자수
0,서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",NaN
1,서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",NaN
2,서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",NaN
3,호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",NaN
4,서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",NaN
...,...,...,...
281,서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",NaN
282,강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",NaN
283,모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",NaN
284,석촌시장,"POLYGON ((14149657.028 4509182.741, 14149713.8...",NaN


#### 상권명을 index로 해서

In [77]:
df_poly_286 = df_poly_286.set_index('상권명')
df_poly_286

,geometry,확진자수
상권명,,
서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",NaN
서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",NaN
서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",NaN
호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",NaN
서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",NaN
...,...,...
서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",NaN
강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",NaN
모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",NaN


In [87]:
df_poly_286['확진자수'] = inf_in_poly['상권명'].value_counts()
df_inf_count = df_poly_286.copy()
df_inf_count = df_inf_count.reset_index()
df_inf_count

,상권명,geometry,확진자수
0,서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",482
1,서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",126
2,서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",196
3,호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",66
4,서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",52
...,...,...,...
281,서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",9
282,강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",45
283,모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",6
284,석촌시장,"POLYGON ((14149657.028 4509182.741, 14149713.8...",6


In [79]:
df_inf_count.확진자수.sort_values(ascending=False) 

상권명
이태원 관광특구           646
홍익대학교 주변           536
서울 관악구 서울대입구역_1    482
학동로56길             471
용산구청               353
                  ... 
남부순환로97길             6
영천시장                 6
남성사계시장               6
영등포전통시장              6
코엑스                  6
Name: 확진자수, Length: 286, dtype: int64

In [80]:
df_inf_count.확진자수.sum() # 가용 확진자 수와 일치 확인.

14623

In [88]:
df_inf_count.to_csv('df_inf_count.csv')

### 분기별로 상권별 확진자 수 구하기

In [68]:
# 일자 포함된 확진자 데이터 불러오기
# 아래의 작업은 이미 했던 작업들. 일자만 포함됨

In [40]:
df =pd.read_csv('./data/infection_data.csv', index_col = 0)

In [41]:
df_in = df[['일자','시','지명','좌표']]
df_in.일자 ='2020/' + df_in.일자 
df_in.일자 = pd.to_datetime(df_in.일자, errors='coerce')
df_in.dropna(inplace=True)
df_seoul = df_in[df_in['시'].str.contains('서울')]
df_seoul

,일자,시,지명,좌표
0,2020-03-03,서울특별시,피자몰회기역점,"37.5900925507337,127.05638644492198"
1,2020-03-03,서울특별시,코로나19경희대학교병원선별진료소,"37.5938752004487,127.051832862704"
51,2020-02-25,서울특별시,긱스타PC방,"37.4798642759744,126.952112617476"
52,2020-02-17,서울특별시,엄마밥상,"37.47155151403081,126.94015305484399"
53,2020-02-18,서울특별시,언제나열린이비인후과의원,"37.48476920038009,126.930088627422"
...,...,...,...,...
1773,2020-06-22,서울특별시,오약국,"37.49910868484411,126.998588658301"
1774,2020-06-22,서울특별시,삼번가우동서래점,"37.49771752481289,126.99814086321199"
1775,2020-06-22,서울특별시,연세박세진의원,"37.4992600536766,126.99857056129102"
1776,2020-06-22,서울특별시,세븐일레븐반포서래점,"37.497735549263446,126.99843941035071"


In [42]:
lat_list=[]
lon_list=[]
 
points = df_seoul.좌표.apply(lambda x : x.split(',')) 

for i in range(len(points)):
        lat = points.values[i][0]
        lon = points.values[i][1]
        
        lat_list.append(float(lat))
        lon_list.append(float(lon))
        
df_seoul['lat'] = lat_list
df_seoul['lon'] = lon_list      

In [43]:
df_seoul = gpd.GeoDataFrame(df_seoul, geometry=gpd.points_from_xy(df_seoul['lon'],df_seoul['lat']))
df_seoul = df_seoul[['일자','지명','geometry']]
df_seoul.rename(columns={'지명':'확진장소'},inplace=True)

In [31]:
df_seoul.head()

,일자,확진장소,geometry
0,2020-03-03,피자몰회기역점,POINT (127.05639 37.59009)
1,2020-03-03,코로나19경희대학교병원선별진료소,POINT (127.05183 37.59388)
51,2020-02-25,긱스타PC방,POINT (126.95211 37.47986)
52,2020-02-17,엄마밥상,POINT (126.94015 37.47155)
53,2020-02-18,언제나열린이비인후과의원,POINT (126.93009 37.48477)


In [69]:
# 상권 데이터 불러오기

In [44]:
df_poly = gpd.read_file('./data/df_poly.geojson')
df_poly

,상권명,geometry
0,낙성대시장,"POLYGON ((126.95719 37.47880, 126.95652 37.477..."
1,봉천제일종합시장,"POLYGON ((126.94472 37.48094, 126.94404 37.481..."
2,도곡시장,"POLYGON ((127.05253 37.49702, 127.05192 37.497..."
3,강남개포시장,"POLYGON ((127.06765 37.48935, 127.06904 37.489..."
4,화곡본동시장,"POLYGON ((126.84379 37.54250, 126.84272 37.543..."
...,...,...
1489,양재역_3,"POLYGON ((127.03268 37.48735, 127.03196 37.487..."
1490,양재역_2,"POLYGON ((127.03805 37.48366, 127.03748 37.483..."
1491,서울 관악구 신림역_1,"POLYGON ((126.92889 37.48178, 126.92910 37.481..."
1492,서울 관악구 신림역_2,"POLYGON ((126.93000 37.48128, 126.92969 37.484..."


In [16]:
from geopandas import sjoin

In [45]:
# 확진장소가 어느 상권에 포함되는지

inf_in_poly_quarter = gpd.sjoin(df_seoul, df_poly, how='inner', op='within')
inf_in_poly_quarter

,일자,확진장소,geometry,index_right,상권명
51,2020-02-25,긱스타PC방,POINT (126.95211 37.47986),1387,서울 관악구 서울대입구역_1
57,2020-02-20,에그드랍서울대입구점,POINT (126.95289 37.48021),1387,서울 관악구 서울대입구역_1
1239,2020-03-09,서울대입구역2호선,POINT (126.95271 37.48128),1387,서울 관악구 서울대입구역_1
1245,2020-03-12,서울대입구역2호선,POINT (126.95271 37.48128),1387,서울 관악구 서울대입구역_1
1280,2020-03-03,스타버스코인노래연습장서울대본점,POINT (126.95298 37.48067),1387,서울 관악구 서울대입구역_1
...,...,...,...,...,...
1672,2020-06-20,봉천지오마트,POINT (126.93847 37.48594),964,봉일시장
1666,2020-06-13,봉천지오마트,POINT (126.93847 37.48594),964,봉일시장
1667,2020-06-15,봉천지오마트,POINT (126.93847 37.48594),964,봉일시장
1668,2020-06-18,봉천지오마트,POINT (126.93847 37.48594),964,봉일시장


In [70]:
inf_in_poly_quarter.info() # 일자가 이미 datetime64

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 14623 entries, 51 to 1672
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   일자           14623 non-null  datetime64[ns]
 1   확진장소         14623 non-null  object        
 2   geometry     14623 non-null  geometry      
 3   index_right  14623 non-null  int64         
 4   상권명          14623 non-null  object        
 5   분기           14623 non-null  int64         
dtypes: datetime64[ns](1), geometry(1), int64(2), object(2)
memory usage: 799.7+ KB


In [46]:
# 분기 칼럼 만들고 분기별 매칭 시키기
inf_in_poly_quarter['분기'] = inf_in_poly_quarter.일자.dt.quarter

In [47]:
inf_in_poly_quarter.head()

,일자,확진장소,geometry,index_right,상권명,분기
51,2020-02-25,긱스타PC방,POINT (126.95211 37.47986),1387,서울 관악구 서울대입구역_1,1
57,2020-02-20,에그드랍서울대입구점,POINT (126.95289 37.48021),1387,서울 관악구 서울대입구역_1,1
1239,2020-03-09,서울대입구역2호선,POINT (126.95271 37.48128),1387,서울 관악구 서울대입구역_1,1
1245,2020-03-12,서울대입구역2호선,POINT (126.95271 37.48128),1387,서울 관악구 서울대입구역_1,1
1280,2020-03-03,스타버스코인노래연습장서울대본점,POINT (126.95298 37.48067),1387,서울 관악구 서울대입구역_1,1


###  분기별로 inf_in_poly_count뒤 concat

In [48]:
inf_in_poly_1 =inf_in_poly_quarter[inf_in_poly_quarter.분기 == 1]
inf_in_poly_2 =inf_in_poly_quarter[inf_in_poly_quarter.분기 == 2]

In [49]:
df_poly_286_1 = df_poly_286.copy()
df_poly_286_1 = df_poly_286_1.set_index('상권명')
df_poly_286_1['확진자수'] = inf_in_poly_1['상권명'].value_counts()
df_poly_286_1['분기'] = 1
df_poly_286_1

,geometry,확진자수,분기
상권명,,,
서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",176.0,1
서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",6.0,1
서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",24.0,1
호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",6.0,1
서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",32.0,1
...,...,...,...
서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",NaN,1
강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",NaN,1
모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",NaN,1


In [50]:
df_poly_286_2 = df_poly_286.copy()
df_poly_286_2 = df_poly_286_2.set_index('상권명')
df_poly_286_2['확진자수'] = inf_in_poly_2['상권명'].value_counts()
df_poly_286_2['분기'] = 2
df_poly_286_2

,geometry,확진자수,분기
상권명,,,
서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",306.0,2
서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",120.0,2
서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",172.0,2
호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",60.0,2
서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",20.0,2
...,...,...,...
서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",9.0,2
강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",45.0,2
모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",6.0,2


In [71]:
# concat

In [60]:
re = pd.concat([df_poly_286_1,df_poly_286_2])
re

,geometry,확진자수,분기
상권명,,,
서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",176.0,1
서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",6.0,1
서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",24.0,1
호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",6.0,1
서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",32.0,1
...,...,...,...
서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",9.0,2
강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",45.0,2
모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",6.0,2


In [61]:
re.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 572 entries, 서울 관악구 서울대입구역_1 to 봉일시장
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  572 non-null    geometry
 1   확진자수      336 non-null    float64 
 2   분기        572 non-null    int64   
dtypes: float64(1), geometry(1), int64(1)
memory usage: 17.9+ KB


In [62]:
re.확진자수.sum() # 기존 inf_in_poly_quarter에서의 확진자 수와 일치 확인

14623.0

In [64]:
re.drop('geometry',axis=1, inplace=True)
re = re.reset_index()
re.head()

,상권명,확진자수,분기
0,서울 관악구 서울대입구역_1,176.0,1
1,서림7길,6.0,1
2,서울 관악구 신림역_4,24.0,1
3,호암로24길,6.0,1
4,서울 관악구 대학동_1,32.0,1


In [66]:
re.to_csv('상권별 확진자수(분기별).csv')

In [ ]:
# inf_in_poly_quarter['분기'] = inf_in_poly_quarter.일자.dt.quarter
# inf_in_poly_1 =inf_in_poly_quarter[inf_in_poly_quarter.분기 == 1]
# inf_in_poly_2 =inf_in_poly_quarter[inf_in_poly_quarter.분기 == 2]
# df_poly_286_1 = df_poly_286.copy()
# df_poly_286_1 = df_poly_286_1.set_index('상권명')
# df_poly_286_1['확진자수'] = inf_in_poly_1['상권명'].value_counts()
# df_poly_286_1['분기'] = 1
# df_poly_286_2 = df_poly_286.copy()
# df_poly_286_2 = df_poly_286_2.set_index('상권명')
# df_poly_286_2['확진자수'] = inf_in_poly_2['상권명'].value_counts()
# df_poly_286_2['분기'] = 2
# re = pd.concat([df_poly_286_1,df_poly_286_2])
# re.drop('geometry',axis=1, inplace=True)
# re = re.reset_index()


### folium.Choropleth

In [89]:
df_inf_count

,상권명,geometry,확진자수
0,서울 관악구 서울대입구역_1,"POLYGON ((14132380.874 4506510.016, 14132472.9...",482
1,서림7길,"POLYGON ((14131122.900 4504965.236, 14130993.2...",126
2,서울 관악구 신림역_4,"POLYGON ((14129905.414 4506849.016, 14129748.6...",196
3,호암로24길,"POLYGON ((14130583.054 4504768.704, 14130554.9...",66
4,서울 관악구 대학동_1,"POLYGON ((14130716.868 4505103.626, 14130753.8...",52
...,...,...,...
281,서울 성북구 성신여대입구역,"POLYGON ((14139744.767 4522081.907, 14139673.3...",9
282,강남구 논현역_1,"POLYGON ((14140302.176 4509665.388, 14139928.3...",45
283,모래내로7길,"POLYGON ((14127759.832 4518632.660, 14127763.5...",6
284,석촌시장,"POLYGON ((14149657.028 4509182.741, 14149713.8...",6


In [119]:
df_inf_count.to_file(driver='GeoJSON', filename='df_inf_count.geojson')

In [ ]:
# 위에 저장

In [139]:
df_inf_count = gpd.GeoDataFrame.from_file('./data/df_inf_count.geojson')
geo = './data/df_poly_286_4326.geojson'
geo2 = json.load(open(geo, encoding='utf-8'))

m = folium.Map(location=[37.51766234549587, 126.95884996942507], zoom_start=12, tiles="cartodbdark_matter")


folium.GeoJson(df_inf_count, name='sang',
               tooltip=folium.features.GeoJsonTooltip(fields=['상권명','확진자수'])).add_to(m)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=geo2,
    data=df_inf_count,
    columns=('상권명', '확진자수'),
    key_on='feature.properties.상권명',
    fill_color='PuRd',
    fill_opacity=0.9,
    legend_name='확진자수',
).add_to(m)

m.save('inf_count.html')
m

In [137]:
df_inf_count = gpd.GeoDataFrame.from_file('./data/df_inf_count.geojson')
geo = './data/df_poly_286_4326.geojson'
geo2 = json.load(open(geo, encoding='utf-8'))


m = folium.Map(location=[37.51766234549587, 126.95884996942507], zoom_start=12, tiles="stamenterrain")


# Choropleth 레이어를 만들고, 맵 m에 추가합니다.

folium.GeoJson(df_inf_count, name='sang',
               tooltip=folium.features.GeoJsonTooltip(fields=['상권명','확진자수'])).add_to(m)

folium.Choropleth(
    geo_data=geo2,
    data=df_inf_count,
    columns=('상권명', '확진자수'),
    key_on='feature.properties.상권명',
    fill_opacity=0.9,
    fill_color='PuRd',
    legend_name='확진자수',
).add_to(m)

m.save('inf_count2.html')
m